In [15]:
import json
import pandas as pd

In [16]:
from datasets import load_dataset

ds = load_dataset("gorpeliates/swe-bench-comments")

In [2]:
#  'dev' split
df = ds['dev'].to_pandas()

df.to_csv("swe_bench_comments_dev.csv", index=False)

In [3]:
df.head(5)

,repo,instance_id,base_commit,patch,test_patch,problem_statement,hints_text,created_at,version,FAIL_TO_PASS,PASS_TO_PASS,environment_setup_commit,comments
0,sqlfluff/sqlfluff,sqlfluff__sqlfluff-4764,a820c139ccbe6d1865d73c4a459945cd69899f8f,diff --git a/src/sqlfluff/cli/commands.py b/sr...,diff --git a/test/cli/commands_test.py b/test/...,Enable quiet mode/no-verbose in CLI for use in...,None,2023-04-16T14:24:42Z,1.40,"[""test/cli/commands_test.py::test__cli__fix_mu...","[""test/cli/commands_test.py::test__cli__comman...",d19de0ecd16d298f9e3bfb91da122734c40c01e5,[]
1,sqlfluff/sqlfluff,sqlfluff__sqlfluff-2862,447ecf862a4d2b977d0add9f444655357b9c4f1f,diff --git a/src/sqlfluff/core/linter/common.p...,diff --git a/test/api/simple_test.py b/test/ap...,fix keep adding new line on wrong place \n### ...,"> Version\r\n> sqlfluff, version 0.6.2\r\n\r\n...",2022-03-14T19:46:08Z,0.10,"[""test/api/simple_test.py::test__api__lint_str...","[""test/api/simple_test.py::test__api__lint_str...",3d52e8270d82aeccf4c516d059a80a6947919aea,['Moved this class so `BaseSegment` can use it...
2,sqlfluff/sqlfluff,sqlfluff__sqlfluff-2336,37a993f7ad841ab3035d1db5ce6525f2e5584fd5,diff --git a/src/sqlfluff/core/rules/analysis/...,diff --git a/test/core/rules/reference_test.py...,L026: Rule incorrectly flag column does not ex...,None,2022-01-17T21:35:10Z,0.80,"[""test/core/rules/reference_test.py::test_obje...",[],a5c4eae4e3e419fe95460c9afd9cf39a35a470c4,['A few small cleanups I made while adapting L...
3,sqlfluff/sqlfluff,sqlfluff__sqlfluff-5074,7b7fd603a19755a9f3707ebbf95d18ee635716d8,diff --git a/src/sqlfluff/core/errors.py b/src...,diff --git a/test/cli/commands_test.py b/test/...,Inconsistent output depending on --processes f...,This is _very_ interesting! I'll pick this one...,2023-08-08T23:31:59Z,2.10,"[""test/cli/commands_test.py::test__cli__comman...","[""test/cli/commands_test.py::test__cli__comman...",7b7fd603a19755a9f3707ebbf95d18ee635716d8,[]
4,sqlfluff/sqlfluff,sqlfluff__sqlfluff-3436,23cd31e77a712a210c734e38488d7a34afd83a25,diff --git a/src/sqlfluff/core/templaters/slic...,diff --git a/test/core/templaters/jinja_test.p...,Fatal templating error with Jinja templater. T...,I'll take a look.\r\n\r\nAnd darn it -- first ...,2022-06-07T21:36:59Z,0.13,"[""test/core/templaters/jinja_test.py::test__te...","[""test/core/templaters/jinja_test.py::test__te...",6e8ce43a4958dbaa56256365c2a89d8db92e07d6,['Unrelated to the fix: This comment was garbl...


In [4]:
sample_df = df.sample(n=20, random_state=42)

In [5]:
sample_df = sample_df.reset_index(drop=True)
sample_df.to_csv("swe_bench_comments_sample.csv", index=False)

In [19]:

df['problem_statement'] = df['problem_statement'].fillna("")
df['hints_text'] = df['hints_text'].fillna("")

labeled_data = []

for _, row in df.iterrows():
    problem = row['problem_statement'].strip()
    hint = row['hints_text'].strip()

    # Problem category
    if "loop limit" in problem.lower():
        problem_category = "Autofix loop limit reached"
    elif "dicom" in problem.lower():
        problem_category = "DICOM file error"
    elif "rename" in problem.lower() or "change" in problem.lower():
        problem_category = "Parameter rename request"
    elif "decode" in problem.lower() or "ambiguous vr" in problem.lower():
        problem_category = "Dataset decoding error"
    elif "attribute" in problem.lower():
        problem_category = "Attribute assignment bug"
    elif "parse" in problem.lower():
        problem_category = "Parsing error"
    elif "missing" in problem.lower() and "example" in problem.lower():
        problem_category = "Missing reproducible example"
    elif "false positive" in problem.lower():
        problem_category = "False positive detection"
    else:
        problem_category = " ".join(problem.split()[:4])

    # Hint type
    if "minimal" in hint.lower() or "example" in hint.lower():
        hint_type = "Minimal reproduction example"
    elif "support" in hint.lower():
        hint_type = "Supportive agreement"
    elif "not much" in hint.lower() or "cannot" in hint.lower():
        hint_type = "No solution possible"
    elif "traceback" in hint.lower():
        hint_type = "Traceback inconclusive"
    elif "patch" in hint.lower() or "pr" in hint.lower():
        hint_type = "Patch or PR merged"
    elif "not reproducible" in hint.lower():
        hint_type = "Not reproducible, closing"
    elif hint.strip() == "":
        hint_type = "No hints provided"
    else:
        hint_type = "General comment"

    if "not reproducible" in hint.lower() or "cannot reproduce" in hint.lower():
        is_misleading = 1
    elif "different" in hint.lower() or "actually" in hint.lower():
        is_misleading = 1
    elif "unsure" in hint.lower():
        is_misleading = "?"
    else:
        is_misleading = 0

    labeled_data.append({
        "problem_category": problem_category,
        "hint_type": hint_type,
        "is_misleading": is_misleading,
    })

output_path = 'swe_bench_comments_labeled.json'
with open(output_path, 'w') as f:
    json.dump(labeled_data, f, indent=2)

output_path


'swe_bench_comments_labeled.json'